In [ ]:
#A = gov
#B = anmawon
#total = union(A,B)
#safe_shop = intersection(total, B)

In [1]:
path_union_shop = "./anmawon/local_update.csv"
path_gov_shop = "./government/manual_edit_government_geolocated.csv"

import pandas as pd
import math

In [3]:

gov_shop = pd.read_csv(path_gov_shop)
anmawon_union_shop = pd.read_csv(path_union_shop)


del anmawon_union_shop["region"]
anmawon_union_shop["checked"] = True


unique_addr_list = list(set(anmawon_union_shop.loc[:, "NAVER_addr"]).union(set(gov_shop.loc[:, "NAVER_addr"])))


In [4]:
final_data_list = []
for unique_addr in unique_addr_list:
    if (gov_shop['NAVER_addr'].eq(unique_addr)).any():
        row = gov_shop.loc[gov_shop['NAVER_addr'] == unique_addr].to_dict(orient="records")[0]
        name, phone, x, y = [row.get(key) for key in ["name", "phone", "x", "y"]]
        # check union(gov, anmawon)
        if (anmawon_union_shop['NAVER_addr'].eq(unique_addr)).any():
            row = anmawon_union_shop.loc[anmawon_union_shop['NAVER_addr'] == unique_addr].to_dict(orient="records")[0]
            link, checked = row["link"], "O"
            if isinstance(phone, float):
                if math.isnan(phone):
                    phone = row["phone"]
        else:
            link, checked = "", "X"

    else:  # only in anmawon_union_shop
        row = anmawon_union_shop.loc[anmawon_union_shop['NAVER_addr'] == unique_addr].to_dict(orient="records")[0]
        name, phone, x, y = [row.get(key) for key in ["name", "phone", "x", "y"]]
        link, checked = row["link"], "???(정부사이트X, 협회사이트O)"

    final_data_list.append(
        {'unique_addr': unique_addr,
        'name': name,
        'phone':phone,
        'x':x ,
        'y':y ,
        'link': link ,
        'checked':checked })

final_data = pd.DataFrame({c: pd.Series(dtype=t) for c, t in {'unique_addr': 'str', 'name': 'str', 'phone': 'str', 'x':"float", 'y':'float', 'checked':'str', 'link':'str'}.items()})

final_data = final_data.append(final_data_list, ignore_index=False, sort=False)


/tmp/ipykernel_300/498848561.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append(final_data_list, ignore_index=False, sort=False)


In [5]:
with open("final_data.json", "w", encoding="utf-8") as f:
    final_data.to_json(f, orient="records", force_ascii=False)

    # then, manually change to .js format as "var data = [....]"